## Implementing a ChatGPT App with LangChain from Scratch

This is part of my **"Learn LangChain, Pinecone & OpenAI: Build Next-Gen LLM Apps"** course.

https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/?referralCode=4B17E3BD4CBBEA3B8321

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#         SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in German.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.run({'content': content})
    print(response)
    print('-' * 50)
    



> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: 


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Adding Chat Memory Using ConversationBufferMemory

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.run({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt: Write the Einstein's equation.
The Einstein's equation is commonly known as E=mc^2, where E represents energy, m represents mass, and c^2 represents the speed of light squared.
--------------------------------------------------
Your prompt: quit
Goodbye!


## Saving Chat Sessions

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.run({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt: quit
Goodbye!


In [ ]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='Bucharest is ...'), AIMessage(content='the capital and largest city of Romania. It is located in the southern part of the country and is known for its rich history, stunning architecture, and vibrant cultural scene.'), HumanMessage(content='Its area is ...'), AIMessage(content='Bucharest has an area of approximately 228 square kilometers (88 square miles).'), HumanMessage(content='What about its population?'), AIMessage(content='As of 2021, the estimated population of Bucharest is around 2.1 million people. It is the most populous city in Romania and serves as an important cultural, economic, and industrial hub for the country.'), HumanMessage(content='paris is ... '), AIMessage(content='Paris is the capital and largest city of France. It is located in the north-central part of the country, along the Seine River. Paris is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is renowned for its art, fashion, cuisine